In [4]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

generations = 500
df = pd.read_csv('sat.trn',header=None,sep = '\s')

X = df.iloc[:,:-1].to_numpy()
y = df.iloc[:,-1].to_numpy()

# 7-> 6 and 1-6 -> 0:5
y[y==7] = 6
for i in range(6):
    y[y == (i + 1)] = i
    
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

def get_winner(class1):
    
    numbers = [*range(6)] # number 2 > number 1
    number_of_classification = len(numbers)

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 4435
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-satellite2")
    [stats, winner] = run(config_path)
    
    return winner

winner_list = []
classes = [[0],[1],[2],[3],[4],[5]]

for class1 in classes:
    winner = get_winner(class1)
    winner_list.append(winner)
    


number of class1:1072
number of class2:3363

 ****** Running generation 0 ****** 

Population's average fitness: 0.62749 stdev: 0.21286
Best fitness: 0.77497 - size: (2, 7) - species 4 - id 102
Average adjusted fitness: 0.513
Mean genetic distance 2.701, standard deviation 0.448
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 10.199 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.64445 stdev: 0.20198
Best fitness: 0.83585 - size: (3, 9) - species 2 - id 267
Average adjusted fitness: 0.546
Mean genetic distance 2.784, standard deviation 0.494
Population of 198 members in 18 species
Total extinctions: 0
Generation time: 11.047 sec (10.623 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.65725 stdev: 0.19380
Best fitness: 0.92830 - size: (3, 10) - species 14 - id 509
Average adjusted fitness: 0.444
Mean genetic distance 2.834, standard deviation 0.501
Population of 199 members in 18 species
Total extin

Population's average fitness: 0.64756 stdev: 0.22687
Best fitness: 0.94228 - size: (6, 12) - species 18 - id 2899
Average adjusted fitness: 0.402
Mean genetic distance 3.071, standard deviation 0.570
Population of 198 members in 21 species
Total extinctions: 0
Generation time: 14.139 sec (14.221 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.67159 stdev: 0.20272
Best fitness: 0.94228 - size: (6, 12) - species 18 - id 2899
Average adjusted fitness: 0.433
Mean genetic distance 3.094, standard deviation 0.554
Population of 200 members in 23 species
Total extinctions: 0
Generation time: 14.446 sec (14.266 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.66869 stdev: 0.20609
Best fitness: 0.94228 - size: (6, 12) - species 18 - id 2899
Average adjusted fitness: 0.429
Mean genetic distance 3.134, standard deviation 0.536
Population of 192 members in 23 species
Total extinctions: 0
Generation time: 14.854 sec (14.437 average)

 

Population's average fitness: 0.65858 stdev: 0.21641
Best fitness: 0.95558 - size: (6, 16) - species 29 - id 5795
Average adjusted fitness: 0.416
Mean genetic distance 3.260, standard deviation 0.545
Population of 196 members in 20 species
Total extinctions: 0
Generation time: 14.004 sec (13.656 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.64571 stdev: 0.23022
Best fitness: 0.95558 - size: (6, 16) - species 29 - id 5795
Average adjusted fitness: 0.438
Mean genetic distance 3.248, standard deviation 0.550
Population of 198 members in 20 species
Total extinctions: 0
Generation time: 13.973 sec (13.777 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.65498 stdev: 0.22304
Best fitness: 0.95558 - size: (6, 16) - species 29 - id 5795
Average adjusted fitness: 0.562
Mean genetic distance 3.243, standard deviation 0.548
Population of 198 members in 21 species
Total extinctions: 0
Generation time: 14.494 sec (13.917 average)

 

Population's average fitness: 0.66069 stdev: 0.22333
Best fitness: 0.95558 - size: (6, 16) - species 29 - id 5795
Average adjusted fitness: 0.400
Mean genetic distance 3.230, standard deviation 0.570
Population of 199 members in 17 species
Total extinctions: 0
Generation time: 15.008 sec (14.840 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.66470 stdev: 0.21206
Best fitness: 0.95558 - size: (6, 16) - species 29 - id 5795
Average adjusted fitness: 0.429
Mean genetic distance 3.248, standard deviation 0.578
Population of 199 members in 17 species
Total extinctions: 0
Generation time: 15.567 sec (14.959 average)

 ****** Running generation 75 ****** 

Population's average fitness: 0.69077 stdev: 0.19378
Best fitness: 0.95558 - size: (6, 16) - species 29 - id 5795
Average adjusted fitness: 0.437
Mean genetic distance 3.236, standard deviation 0.571
Population of 199 members in 17 species
Total extinctions: 0
Generation time: 15.219 sec (15.023 average)

 

Population's average fitness: 0.64247 stdev: 0.23420
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.401
Mean genetic distance 3.174, standard deviation 0.589
Population of 202 members in 14 species
Total extinctions: 0
Generation time: 19.657 sec (18.298 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.67927 stdev: 0.20448
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.431
Mean genetic distance 3.143, standard deviation 0.576
Population of 203 members in 13 species
Total extinctions: 0
Generation time: 18.911 sec (18.421 average)

 ****** Running generation 99 ****** 

Population's average fitness: 0.64883 stdev: 0.23505
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.461
Mean genetic distance 3.130, standard deviation 0.578
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 18.711 sec (18.477 average)

Population's average fitness: 0.71283 stdev: 0.17613
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.446
Mean genetic distance 2.988, standard deviation 0.520
Population of 201 members in 17 species
Total extinctions: 0
Generation time: 20.596 sec (19.202 average)

 ****** Running generation 122 ****** 

Population's average fitness: 0.68825 stdev: 0.19997
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.442
Mean genetic distance 2.976, standard deviation 0.531
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 19.875 sec (19.560 average)

 ****** Running generation 123 ****** 

Population's average fitness: 0.66492 stdev: 0.21842
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.424
Mean genetic distance 3.022, standard deviation 0.519
Population of 194 members in 20 species
Total extinctions: 0
Generation time: 19.944 sec (19.860 averag

Population's average fitness: 0.65870 stdev: 0.22396
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.404
Mean genetic distance 3.000, standard deviation 0.512
Population of 197 members in 17 species
Total extinctions: 0
Generation time: 19.414 sec (18.405 average)

 ****** Running generation 146 ****** 

Population's average fitness: 0.65732 stdev: 0.22755
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.410
Mean genetic distance 2.990, standard deviation 0.504
Population of 195 members in 16 species
Total extinctions: 0
Generation time: 22.110 sec (18.817 average)

 ****** Running generation 147 ****** 

Population's average fitness: 0.66505 stdev: 0.22031
Best fitness: 0.95603 - size: (8, 21) - species 15 - id 15770
Average adjusted fitness: 0.418
Mean genetic distance 2.990, standard deviation 0.496
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 24.959 sec (19.563 averag

Population's average fitness: 0.65086 stdev: 0.24488
Best fitness: 0.95671 - size: (9, 21) - species 15 - id 25439
Average adjusted fitness: 0.408
Mean genetic distance 3.000, standard deviation 0.547
Population of 196 members in 15 species
Total extinctions: 0
Generation time: 23.008 sec (23.551 average)

 ****** Running generation 170 ****** 

Population's average fitness: 0.66428 stdev: 0.23297
Best fitness: 0.95671 - size: (9, 21) - species 15 - id 25439
Average adjusted fitness: 0.419
Mean genetic distance 3.003, standard deviation 0.547
Population of 198 members in 16 species
Total extinctions: 0
Generation time: 23.450 sec (23.225 average)

 ****** Running generation 171 ****** 

Population's average fitness: 0.66356 stdev: 0.23646
Best fitness: 0.95671 - size: (9, 21) - species 15 - id 25439
Average adjusted fitness: 0.421
Mean genetic distance 3.005, standard deviation 0.546
Population of 200 members in 16 species
Total extinctions: 0
Generation time: 22.978 sec (23.137 averag

KeyboardInterrupt: 

In [ ]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-satellite2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

test_df = pd.read_csv('sat.tst',header=None,sep = '\s')

testX = test_df.iloc[:,:-1].to_numpy()
testy = test_df.iloc[:,-1].to_numpy()

# 7-> 6 and 1-6 -> 0:5
testy[testy==7] = 6
for i in range(6):
    testy[testy == (i + 1)] = i
    
def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(6)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(2000):
    pred = []
    for i in range(6):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    pred_array = np.array(pred).reshape(6,)
    #print(pred_array)
    if (np.sum(pred_array) >= 1):
        pred_value.append(np.where(pred_array==1))
    else:
        pred_value.append(np.array(range(6)))

    #print(vote)

import random

list_P = []
for i in range(2000):
    if (len(pred_value[i])== 1):
        #print(pred_value[i][0], digits.target[1617+i])
        list_P.append(random.sample(pred_value[i][0].tolist(),1)[0])
    else:
        list_P.append(random.sample(pred_value[i].tolist(),1)[0])

test_acc = np.sum(list_P == testy)/2000
print("Test Accuracy:{}".format(test_acc))

def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(6)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(4435):
    pred = []
    for i in range(6):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    pred_array = np.array(pred).reshape(6,)
    #print(pred_array)
    if (np.sum(pred_array) >= 1):
        pred_value.append(np.where(pred_array==1))
    else:
        pred_value.append(np.array(range(6)))

import random

list_P = []
for i in range(4435):
    if (len(pred_value[i])== 1):
        #print(pred_value[i][0], digits.target[1617+i])
        list_P.append(random.sample(pred_value[i][0].tolist(),1)[0])
    else:
        list_P.append(random.sample(pred_value[i].tolist(),1)[0])
train_acc = np.sum(list_P == y)/4435
print("Train Accuracy:{}".format(train_acc))

winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
avg_train_acc = np.mean(winner_fitness)
print("Avg Train Accuracy:{}".format(avg_train_acc))


# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
total_nodes = np.sum(list_nodes_number)
total_connections = np.sum(list_connection_number)
print("Total nodes:{} Total connections::{}".format(total_nodes, total_connections))

with open("ova_1.txt","w") as f:
    for item in [avg_train_acc, train_acc, test_acc, total_nodes, total_connections]:
        f.write(str(item)+'\n')